# 8 - Classes and Objects

## Changing the String Representation of Instances
To change the string representation of an instance, define the __str__() and __repr__() methods.


In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
 
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)


In [2]:
pair = Pair(1, 2)

In [3]:
pair  # from __repr__

Pair(1, 2)

In [4]:
str(pair)  # from __str__

'(1, 2)'

## Customizing String Formatting
To customize string formatting, define the __format__() method on a class.

In [8]:
_formats = {
 'ymd' : '{d.year}-{d.month}-{d.day}',
 'mdy' : '{d.month}/{d.day}/{d.year}',
 'dmy' : '{d.day}/{d.month}/{d.year}'
 }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)


In [9]:
d = Date(2012, 12, 21)

In [10]:
format(d)

'2012-12-21'

In [11]:
format(d, 'mdy')

'12/21/2012'

## Making Objects Support the Context-Management Protocol
In order to make an object compatible with the with statement, you need to implement __enter__() and __exit__() methods. 

In [12]:
class MyClass:
    def __init__(self):
        print("__init__")

    def __enter__(self): 
        print("__enter__")

    def __exit__(self, type, value, traceback):
        print("__exit__")

    def __del__(self):
        print("__del__")

with MyClass(): 
    print("body")


__init__
__enter__
body
__exit__
__del__


## Saving Memory When Creating a Large Number of Instances
For classes that primarily serve as simple data structures, you can often greatly reduce the memory footprint of instances by adding the __slots__ attribute to the class definition. When you define __slots__, Python uses a much more compact internal representation for instances. 

In [ ]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day


## Encapsulating Names in a Class
One convention is that any name that starts with a single leading underscore (_) should always be assumed to be internal implementation. 

In [13]:
class A:
    def __init__(self):
        self._internal = 0  # An internal attribute
        self.public = 1     # A public attribute

    def public_method(self):
        pass
    
    def _internal_method(self):
        pass


Python doesn’t actually prevent someone from accessing internal names. However, doing so is considered impolite, and may result in fragile code. You may also encounter the use of two leading underscores (__) on names within class definitions. The use of double leading underscores causes the name to be mangled to something else.

In [14]:
class SomeClass:
    x = 1
    __y = 2
    def __init__(self, a, b):
        self.a = a
        self.__b = b


In [15]:
vars(SomeClass)  # notice the class attribute _SomeClass__y

mappingproxy({'__module__': '__main__',
              'x': 1,
              '_SomeClass__y': 2,
              '__init__': <function __main__.SomeClass.__init__(self, a, b)>,
              '__dict__': <attribute '__dict__' of 'SomeClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'SomeClass' objects>,
              '__doc__': None})

In [17]:
vars(SomeClass(5,6))  # and the instance attribute _SomeClass__b

{'a': 5, '_SomeClass__b': 6}

For most code, you should probably just make your nonpublic names start with a single underscore. If, however, you know that your code will involve subclassing, and there are internal attributes that should be hidden from subclasses, use the double underscore instead.

## Creating Managed Attributes
A simple way to customize access to an attribute is to define it as a property. 

In [18]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
 
    # Getter function
    @property
    def first_name(self):
        return self._first_name
 
    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
 
    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")


In [25]:
a = Person("Guido")

In [26]:
a.first_name

'Guido'

In [27]:
a.first_name = 42

TypeError: Expected a string

In [28]:
del a.first_name

AttributeError: Can't delete attribute

In [29]:
del a  # can still delete the object though

In [30]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, first_name)>,
              'first_name': <property at 0x2acaaa15db8>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [32]:
Person.__dict__["first_name"]

Properties can also be a way to define computed attributes.

In [34]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
 
    @property
    def area(self):
        return math.pi * self.radius ** 2
 
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius


In [36]:
c = Circle(2)

In [38]:
c.area

12.566370614359172

In [39]:
c.perimeter

12.566370614359172

Also, don’t write Python code that features a lot of repetitive property definitions. Code repetition leads to bloated, error prone, and ugly code. As it turns out, there are much better ways to achieve the same thing using descriptors or closures. 

## Calling a Method on a Parent Class
You want to invoke a method in a parent class in place of a method that has been overridden in a subclass.

In [40]:
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam() # Call parent spam()


In [41]:
a = A()
b = B()

In [42]:
b.spam()

B.spam
A.spam


A very common use of super() is in the handling of the __init__() method to make
sure that parents are properly initialized:

In [54]:
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1


In [55]:
B.__mro__

(__main__.B, __main__.A, object)

## Extending a Property in a Subclass

In [1]:
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value

    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")


Extend the name property with new functionality.

In [2]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name

    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)
 
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)


In [3]:
print(super(SubPerson, SubPerson))

<super: <class 'SubPerson'>, <SubPerson object>>


In [4]:
s = SubPerson("Bob")

Setting name to Bob


In [5]:
s.name = "Alex"

Setting name to Alex


## Creating a New Kind of Class or Instance Attribute.
If you want to create an entirely new kind of instance attribute, define its functionality in the form of a descriptor class.

In [7]:
class Integer:
    def __init__(self, name):
        self.name = name
 
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
 
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value
 
    def __delete__(self, instance):
        del instance.__dict__[self.name]


To use a descriptor, instances of the descriptor are placed into a class definition as class variables.

In [8]:
class Point:
    x = Integer('x')
    y = Integer('y')

    def __init__(self, x, y):
        self.x = x
        self.y = y


In [9]:
p = Point(2, 3)

Here is some more advanced descriptor-based code involving a class decorator.

In [10]:
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected ' + str(self.expected_type))
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]


In [11]:
# class decorator that applies it to selected attributes
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            # attach a typed descriptor to the class
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate


In [12]:
# Example use
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


In [13]:
stock = Stock("AAPL", 1000, 198.00)

In [14]:
stock = Stock("AAPL", 1000, 198)

TypeError: Expected <class 'float'>

## Using Lazily Computed Properties

In [15]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
 
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value


In [16]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
 
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius


In [22]:
c = Circle(2)
vars(c)

{'radius': 2}

In [23]:
c.area

Computing area


12.566370614359172

In [24]:
vars(c)

{'radius': 2, 'area': 12.566370614359172}

In [25]:
c.area

12.566370614359172

In [26]:
c.perimeter

Computing perimeter


12.566370614359172

In [27]:
vars(c)

{'radius': 2, 'area': 12.566370614359172, 'perimeter': 12.566370614359172}

One possible downside to this recipe is that the computed value becomes mutable after it’s created.

## Simplifying the Initialization of Data Structures

In [28]:
class Structure:
    _fields= []
 
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
 
        for name, value in zip(self._fields, args):
            setattr(self, name, value)


In [29]:
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x','y']
 
class Circle(Structure):
    _fields = ['radius']
 
def area(self):
    return math.pi * self.radius ** 2


In [31]:
s = Stock('ACME', 50, 91.1)
p = Point(2, 3)
c = Circle(4.5)

In [32]:
s2 = Stock('ACME', 50)

TypeError: Expected 3 arguments

This technique of defining a general purpose __init__() method can be extremely useful if you’re ever writing a program built around a large number of small data structures. 

## Defining an Interface or Abstract Base Class
A major use of abstract base classes is in code that wants to enforce an expected programming interface.

In [1]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
 
    @abstractmethod
    def write(self, data):
        pass


In [2]:
a = IStream()  # abstract classes cannot be instantiated

TypeError: Can't instantiate abstract class IStream with abstract methods read, write

In [3]:
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass

    def write(self, data):
        pass


In [5]:
stream = SocketStream()  # better

We can check that objects support this interface before using them.

In [6]:
if not isinstance(stream, IStream):
    raise TypeError('Expected an IStream')


ABCs also allow other classes to be registered as implementing the required interface. 

In [8]:
import io

# register the built-in I/O classes as supporting our interface
IStream.register(io.IOBase)

f = open("sample.txt")
isinstance(f, IStream)

True

In [9]:
f.close()

It should be noted that @abstractmethod can also be applied to static methods, class methods, and properties.

In [10]:
from abc import ABCMeta, abstractmethod

class A(metaclass=ABCMeta):
    @property
    @abstractmethod
    def name(self):
        pass
 
    @name.setter
    @abstractmethod
    def name(self, value):
        pass
 
    @classmethod
    @abstractmethod
    def method1(cls):
        pass

    @staticmethod
    @abstractmethod
    def method2():
        pass


You can use the predefined ABCs to perform more generalized kinds of type checking.

In [12]:
import collections

x = None

if isinstance(x, collections.Sequence):
    pass

if isinstance(x, collections.Iterable):
    pass

if isinstance(x, collections.Sized):
    pass

if isinstance(x, collections.Mapping):
    pass


It should be noted that, as of this writing, certain library modules don’t make use of these predefined ABCs as you might expect.

In [13]:
from decimal import Decimal
import numbers

x = Decimal('3.4')
isinstance(x, numbers.Real)

False

In [14]:
isinstance(3.14, numbers.Real)

True

In [15]:
isinstance(3, numbers.Real)

True

See [numbers — Numeric abstract base classes](https://docs.python.org/3/library/numbers.html)

Although ABCs facilitate type checking, it’s not something that you should overuse in a program. At its heart, Python is a dynamic language that gives you great flexibility. Trying to enforce type constraints everywhere tends to result in code that is more complicated than it needs to be. You should embrace Python’s flexibility.

## Implementing a Data Model or Type System